# LE1 Performance

In [1]:
#from numba import jit, cuda
#import numpy as np
## to measure exec time
#from timeit import default_timer as timer
#
## normal function to run on cpu
#def func(a):								
#	for i in range(10000000):
#		a[i]+= 1	
#
## function optimized to run on gpu
#@jit(target_backend='cuda')						
#def func2(a):
#	for i in range(10000000):
#		a[i]+= 1
#if __name__=="__main__":
#	n = 10000000							
#	a = np.ones(n, dtype = np.float64)
#	
#	start = timer()
#	func(a)
#	print("without GPU:", timer()-start)	
#	
#	start = timer()
#	func2(a)
#	print("with GPU:", timer()-start)

In [2]:
##Output:
#without GPU: 8.985259440999926
#with GPU: 1.4247172560001218

Anhand des Outputs erkennt man, dass es mit GPU bis zu 6-Mal schneller ist.

# Dashboard für LE2 - LE4

# Import libraries

In [3]:
import pandas as pd
import numpy as np
import sqlalchemy
import sqlite3
import scipy.stats as stats
import panel as pn
from panel.interact import interact
from scipy.stats import linregress
pn.extension('tabulator')
pn.extension()
import hvplot.pandas

# Get Data


In [4]:
# Open connection with sqlit3 --> Databases is automaticly created if not initializes already
con = sqlite3.connect("./data/Klimadaten.db")
# Create cursor for executables
cur = con.cursor()
# Create database engine
engine = sqlalchemy.create_engine("sqlite:///./data/Klimadaten.db")

In [5]:
# list table names
query = f"""
        SELECT name 
        FROM sqlite_master
        WHERE TYPE='table';
        """

cur.execute(query)

# fetch tablenames
table_names = cur.fetchall()

# print examples:
for table in table_names:
        # print tablename
        print(table[0], ":")
        
        # print example 
        query = f"""
        SELECT *
        FROM {table[0]}
        LIMIT 4;
        """
        example = pd.read_sql(query, engine)
        
        print(example.head())

z_agriculture :
    ID  timestamp  country  measurement    detrend  zdetrend
0    1       1965  Belgium        519.8  68.858688  1.671917
1   39       1966  Belgium        507.1  59.653972  1.448423
2   77       1967  Belgium        490.9  46.949256  1.139947
3  115       1968  Belgium        485.7  45.244540  1.098556
rainflux_z :
   ID        date    lat   lon   country  measurement  zmeasurement
0   1  2010-01-01  38.75 -9.75  Portugal     0.000100     -0.459792
1   2  2010-01-01  39.00 -9.75  Portugal     0.000320     -0.191452
2   3  2010-01-01  39.25 -9.75  Portugal     0.000189     -0.390452
3   4  2010-01-01  39.50 -9.75  Portugal     0.000215     -0.374701
grid :
     lat   lon   Country
0  39.50 -9.75  Portugal
1  39.25 -9.75  Portugal
2  39.00 -9.75  Portugal
3  38.75 -9.75  Portugal


In [6]:
# query grid data
query = f"""
        SELECT *
        FROM grid
        """
grid = pd.read_sql(query, engine)

# look at grid
grid.head(4)

,lat,lon,Country
0,39.50,-9.75,Portugal
1,39.25,-9.75,Portugal
2,39.00,-9.75,Portugal
3,38.75,-9.75,Portugal


In [7]:
# get countries
countries = list(grid.Country.unique())

In [8]:
# query agriculture data
query = f"""
        SELECT *
        FROM z_agriculture
        """
agriculture_z = pd.read_sql(query, engine)
agriculture_z.replace(['Portugal', 'Spain', 'Norway', 'Italy', 'Sweden'], countries, inplace = True)
agriculture_z.rename(columns={'timestamp': 'year', 'measurement': 'production [1000t]'}, inplace = True)
agriculture_z.drop(columns = ['ID'], inplace = True)
agriculture_z.year = agriculture_z.year.astype('str')
agriculture_z['production [1000t]'] = agriculture_z['production [1000t]'].fillna(0)
# look at agriculture
agriculture_z.head(4)

,year,country,production [1000t],detrend,zdetrend
0,1965,Belgium,519.8,68.858688,1.671917
1,1966,Belgium,507.1,59.653972,1.448423
2,1967,Belgium,490.9,46.949256,1.139947
3,1968,Belgium,485.7,45.244540,1.098556


In [9]:
query2 = """
SELECT *
FROM z_agriculture
WHERE timestamp BETWEEN 1980 AND 2020
;
"""
agriculture = pd.read_sql(query2,engine)
agriculture

,ID,timestamp,country,measurement,detrend,zdetrend
0,571,1980,Belgium,390.7,-7.812053,-0.189680
1,609,1981,Belgium,371.7,-23.316769,-0.566141
2,647,1982,Belgium,361.3,-30.221485,-0.733790
3,685,1983,Belgium,373.4,-14.626201,-0.355130
4,723,1984,Belgium,360.5,-24.030918,-0.583481
...,...,...,...,...,...,...
2560,972,2016,Turkey,1080.0,41.484129,NaN
2561,1008,2017,Turkey,1165.0,92.887509,NaN
2562,1044,2018,Turkey,1225.0,119.290889,NaN
2563,1080,2019,Turkey,1288.0,148.694269,NaN


In [10]:
# 1min
# query rainflux data
query = f"""
        SELECT * 
        FROM rainflux_z
        """

rainflux_z = pd.read_sql(query, engine)
rainflux_z.date = pd.to_datetime(rainflux_z.date)
rainflux_z['year'] = rainflux_z.date.dt.strftime('%Y')
rainflux_z['year'] = rainflux_z['year'].astype('int')
years = rainflux_z.date.dt.strftime("%Y").unique()
rainflux_z.head(4)

,ID,date,lat,lon,country,measurement,zmeasurement,year
0,1,2010-01-01,38.75,-9.75,Portugal,0.000100,-0.459792,2010
1,2,2010-01-01,39.00,-9.75,Portugal,0.000320,-0.191452,2010
2,3,2010-01-01,39.25,-9.75,Portugal,0.000189,-0.390452,2010
3,4,2010-01-01,39.50,-9.75,Portugal,0.000215,-0.374701,2010


In [11]:
query2 = """
SELECT strftime('%Y', date) AS date,strftime('%m', date) AS month, country, SUM(measurement) AS rainfall,lat,lon
FROM rainflux_z
GROUP BY strftime('%Y-%m', date), country;
"""
rainflux_month = pd.read_sql(query2, engine)
rainflux_month

,date,month,country,rainfall,lat,lon
0,2010,01,España,8.906281,42.75,-9.50
1,2010,01,Italia,10.392260,45.50,7.50
2,2010,01,Norge,2.296230,61.50,4.25
3,2010,01,Portugal,3.150339,38.75,-9.75
4,2010,01,Sverige,0.053376,58.50,10.75
...,...,...,...,...,...,...
655,2020,12,España,8.423653,42.75,-9.50
656,2020,12,Italia,16.498457,45.00,6.75
657,2020,12,Norge,5.979299,61.75,4.25
658,2020,12,Portugal,2.245834,38.75,-9.75


# Dashboard Widgets

In [12]:
multi_country_selector = pn.widgets.CheckButtonGroup(   name='Country selector', 
                                                        value=['all'], 
                                                        options=countries + ['all'])
multi_country_selector2 = pn.widgets.CheckButtonGroup(   name='Country selector', 
                                                        value=['Portugal'], 
                                                        options=countries)

grenze_slider = pn.widgets.FloatSlider( name='Auswahl Grenze(z)', 
                                        start=2, 
                                        end=10, 
                                        step=0.1, 
                                        value=4)

year_slider = pn.widgets.IntSlider( name='Auswahl Jahr', 
                                    start=2010, 
                                    end=2020, 
                                    value=2010)

radio_group = pn.widgets.RadioButtonGroup(  name='Select measurement', 
                                            options=[   'Agriculture_zdetrend',    
                                                        'Rainfall_Magnitude'], 
                                            button_type='success')
start_year_slider = pn.widgets.IntSlider( name = 'Ausgewählt ',
                                    start = 2010,
                                    end = 2020,
                                    step = 1,
                                    value = 2010)
end_year_slider = pn.widgets.IntSlider( name = 'Ausgewählt ',
                                    start = 2010,
                                    end = 2020,
                                    step = 1,
                                    value = 2020)

# Zusammenhangsanalyse

In [13]:
agriculture_z_a = pd.DataFrame(columns = agriculture_z.columns)

## detrend & z-scores agriculture
for c in countries:
    # filter for country
    tmp = agriculture_z[agriculture_z.country == c]
    tmp = tmp[tmp.year.astype('int') >= 2010]
    tmp = tmp[tmp.year.astype('int') <= 2020]
    tmp['fit'] = tmp.year.astype('int') * 0

    values = tmp['production [1000t]']
    dates = tmp['year'].astype('int')

    # fit line (linear regression 1 degree)
    b1_fit, b0_fit = np.polyfit(x = dates, y = values, deg = 1)
    y_fit = []
    for e in dates:
            y_fit.append(b0_fit + b1_fit * e)

    # calculate detrended data
    tmp['detrend'] = values - y_fit
    tmp['fit'] = y_fit

    # calculate z-scores
    tmp["zdetrend"] = stats.zscore(tmp['detrend'])

    # append to df
    agriculture_z_a = agriculture_z_a.append(tmp, ignore_index=True)

# filter for country
tmp = agriculture_z
tmp = tmp[tmp.year.astype('int') >= 2010]
tmp = tmp[tmp.year.astype('int') <= 2020]
tmp['fit'] = tmp.year.astype('int') * 0
tmp = tmp.groupby('year').sum()
tmp.reset_index(inplace = True)
tmp['country'] = 'all'

values = tmp['production [1000t]']
dates = tmp['year'].astype('int')

# fit line (linear regression 1 degree)
b1_fit, b0_fit = np.polyfit(x = dates, y = values, deg = 1)
y_fit = []
for e in dates:
    y_fit.append(b0_fit + b1_fit * e)

# calculate detrended data
tmp['detrend'] = values - y_fit
tmp['fit'] = y_fit

# calculate z-scores
tmp["zdetrend"] = stats.zscore(tmp['detrend'])

# append to df
agriculture_z_a = agriculture_z_a.append(tmp, ignore_index=True)

agriculture_z_a.head(4)


/var/folders/5l/8fvb5kzs1_s2cq12n2mb3tlw0000gn/T/ipykernel_14872/2141471517.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  agriculture_z_a = agriculture_z_a.append(tmp, ignore_index=True)
/var/folders/5l/8fvb5kzs1_s2cq12n2mb3tlw0000gn/T/ipykernel_14872/2141471517.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  agriculture_z_a = agriculture_z_a.append(tmp, ignore_index=True)
/var/folders/5l/8fvb5kzs1_s2cq12n2mb3tlw0000gn/T/ipykernel_14872/2141471517.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  agriculture_z_a = agriculture_z_a.append(tmp, ignore_index=True)
/var/folders/5l/8fvb5kzs1_s2cq12n2mb3tlw0000gn/T/ipykernel_14872/2141471517.py:28: FutureWarning: The frame.append method is deprecated and will be rem

,year,country,production [1000t],detrend,zdetrend,fit
0,2010,Portugal,305.34,144.373864,1.112926,160.966136
1,2011,Portugal,284.22,127.693364,0.984342,156.526636
2,2012,Portugal,288.97,136.882864,1.055181,152.087136
3,2013,Portugal,315.1,167.452364,1.29083,147.647636


In [14]:
## 5min
## set up empty dataframe for corelation data
corr = pd.DataFrame(columns = [ 'grenze_starkregen', 
                                '%-Strakregen', 
                                'Starkregen Anzahl', 
                                'min_Starkregen', 
                                'max_Starkregen', 
                                'avg_Starkregen',
                                'country', 
                                'r-value', 
                                'p-value'])

corr.reset_index(drop=True, inplace=True)

## set up empty dataframe for yearly data
yearly_vals = pd.DataFrame(columns= ['country',
                                    'year',
                                    'yearly_sum',
                                    'production [1000t]',
                                    'detrend',
                                    'zdetrend',
                                    'fit',
                                    'grenze_starkregen'])

for i in np.round(np.linspace(start = 2, stop = 10, num = 81), 1):
    ## setze Starkregen grenze
    grenze_starkregen = i

    ## berechne wie viele % als Starkregen gelten
    query = f"""
        SELECT country, COUNT() AS c
        FROM rainflux_z
        WHERE zmeasurement - {grenze_starkregen} > 0
        GROUP BY country
        ;
        """
    count_1 = pd.read_sql(query, engine)

    query = f"""
        SELECT country, COUNT() AS c
        FROM rainflux_z
        GROUP BY country
        ;
        """
    count_2 = pd.read_sql(query, engine)

    # lese Starkregen magnitude aus
    query = f"""
            SELECT  country, 
                    year, 
                    AVG(yearly_sum) AS yearly_sum, 
                    MIN(min_measurement) AS min_measurement,
                    MAX(max_measurement) AS max_measurement,
                    SUM(sum_measurement) AS sum_measurement,
                    SUM(count_measurement) AS count_measurement
            FROM    (
                    SELECT      country, 
                                strftime('%Y', date) AS year, 
                                SUM(magnitude) AS yearly_sum, 
                                MIN(measurement) AS min_measurement,
                                SUM(measurement) AS sum_measurement,
                                COUNT(measurement) AS count_measurement,
                                MAX(measurement) AS max_measurement 
                    FROM        (
                                SELECT *, zmeasurement - {grenze_starkregen} AS magnitude
                                FROM rainflux_z
                                WHERE zmeasurement - {grenze_starkregen} > 0) 
                    GROUP BY lat, lon, strftime('%Y', date))
            GROUP BY country, year;
        """
    starkregen = pd.read_sql(query, engine)
    ## get values for all
    tmp = starkregen.groupby('year').sum()
    tmp['country'] = 'all'
    tmp['yearly_sum'] = starkregen.groupby('year').mean().yearly_sum
    tmp['min_measurement'] = starkregen.groupby('year').min().min_measurement
    tmp['max_measurement'] = starkregen.groupby('year').max().max_measurement
    tmp.reset_index(inplace=True)

    starkregen = pd.concat([starkregen, tmp]) 

    ## Joine Starkregenfälle mit agricultur daten
    data =starkregen.merge( agriculture_z_a,
                            on = ['country', 'year'],
                            how='left', )
    data['grenze_starkregen'] = grenze_starkregen


    for c in countries:
        ####################################### append correlation data for single country ######################################
        # filter for country
        tmp = data[data.country == c]

        # calculate correlation
        tmp_corr = linregress(tmp.yearly_sum.astype('float'), tmp.zdetrend.astype('float'), alternative= 'two-sided')

        # append to corr df 
        tmp = pd.DataFrame({    'grenze_starkregen' : grenze_starkregen, 
                                '%-Strakregen': count_1[count_1.country == c]['c'] / count_2[count_2.country == c]['c'] * 100, 
                                'Starkregen Anzahl' : count_1[count_1.country == c]['c'], 
                                'min_Starkregen' : tmp.min_measurement.min() * 86400,
                                'max_Starkregen' : tmp.max_measurement.max() * 86400,
                                'avg_Starkregen' : tmp.sum_measurement.sum() / tmp.count_measurement.sum() * 86400, 
                                'country' : c , 
                                'r-value' : tmp_corr[2], 
                                'p-value' : tmp_corr[3]})

        corr = pd.concat([corr, tmp])

    ####################################### append yearly vals ########################################
    tmp = data[yearly_vals.columns]

    yearly_vals = pd.concat([yearly_vals, tmp])
    
    ####################################### append correlation data for overall ######################################    
    tmp = data
    # calculate correlation
    tmp_corr = linregress(tmp.yearly_sum.astype('float'), tmp.zdetrend.astype('float'), alternative= 'two-sided')

    # append to corr df
    tmp = pd.DataFrame({    'grenze_starkregen' : [grenze_starkregen], 
                            '%-Strakregen': [count_1.c.sum() / count_2.c.sum() * 100], 
                            'Starkregen Anzahl' : [count_1.c.sum()], 
                            'min_Starkregen' : [tmp.min_measurement.min() * 86400],
                            'max_Starkregen' : [tmp.max_measurement.max() * 86400],
                            'avg_Starkregen' : [tmp.sum_measurement.sum() / tmp.count_measurement.sum() * 86400], 
                            'country' : ["all" ], 
                            'r-value' : [tmp_corr[2]], 
                            'p-value' : [tmp_corr[3]]})

    corr = pd.concat([corr, tmp])

# convert to int
corr['Starkregen Anzahl'] = corr['Starkregen Anzahl'].astype('int')
corr['%-Strakregen'] = corr['%-Strakregen'].astype('float')

In [15]:
corr.head(4)

,grenze_starkregen,%-Strakregen,Starkregen Anzahl,min_Starkregen,max_Starkregen,avg_Starkregen,country,r-value,p-value
3,2.0,5.459580,18434,10.383005,1795.700305,387.84507,Portugal,-0.007623,0.973142
0,2.0,5.340726,93197,3.343736,3093.187874,338.578392,España,0.000271,0.999046
2,2.0,5.256681,143317,0.81448,2007.821284,269.744956,Norge,-0.003006,0.989408
1,2.0,5.247709,84618,0.512208,3675.396481,459.247759,Italia,0.01511,0.946791


In [16]:
yearly_vals.head(4)

,country,year,yearly_sum,production [1000t],detrend,zdetrend,fit,grenze_starkregen
0,España,2010,14.265255,6039.89,2729.036136,0.971548,3310.853864,2.0
1,España,2010,14.265255,507.49,-2803.363864,-0.998009,3310.853864,2.0
2,España,2011,10.303402,5985.49,2674.559636,0.952154,3310.930364,2.0
3,España,2011,10.303402,520.29,-2790.640364,-0.993479,3310.930364,2.0


# Plots Erstellung

In [17]:
# make dataframe interactive
i_corr = corr.interactive()

yearly_vals['Agriculture_zdetrend'] = yearly_vals.zdetrend
yearly_vals['Rainfall_Magnitude'] = yearly_vals.yearly_sum
i_yearly_vals = yearly_vals.interactive()

yearly_vals2 = yearly_vals[yearly_vals.grenze_starkregen == 2.0]
yearly_vals2 = yearly_vals2.copy()
yearly_vals2['Produktion z(detrend)'] = 0
i_yearly_vals2 = yearly_vals2.interactive()

i_rain = rainflux_z.interactive()
i_agriculture = agriculture_z.drop(columns = ['detrend', 'zdetrend']).interactive()

In [18]:
# generate data pipeline
corr_pipeline = (
    i_corr[ 
        (i_corr.country.isin(multi_country_selector))
    ]
    .sort_values(by='grenze_starkregen')  
    .reset_index(drop=True)
)

vals_pipeline = (
    i_yearly_vals[ 
        (i_yearly_vals.grenze_starkregen == grenze_slider) &
        (i_yearly_vals.year.astype('int') == year_slider)
    ])

vals2_pipeline = (
    i_yearly_vals2[ 
        (i_yearly_vals2.country.isin(multi_country_selector))
    ])

#rain_pipeline = (
#    i_rain[ 
#        (i_rain.country.isin(multi_country_selector))
#    ])
agriculture_pipeline = (
    i_agriculture[
        (i_agriculture.year.astype('int') >= start_year_slider) & 
        (i_agriculture.year.astype('int') <= end_year_slider) & 
        (i_agriculture.country.isin(multi_country_selector2))
    ]
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

rain_pipeline_yearly = (
    i_rain[
        (i_rain.year >= start_year_slider) & 
        (i_rain.year <= end_year_slider) & 
        (i_rain.country.isin(multi_country_selector2))
    ]
    .groupby(['country', 'year']).measurement.sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [19]:
plot1 = corr_pipeline.hvplot(   x = 'grenze_starkregen', 
                                y = 'avg_Starkregen',
                                by = 'country', 
                                line_width = 2, 
                                title = "Einfluss Grenze(z) auf durchschnittlicher Starkregenfall",
                                ylabel= 'Durchschnittlicher Starkregen',
                                xlabel= 'Grenze Starkregen(z)')
plot1

BokehModel(combine_events=True, render_bundle={'docs_json': {'a3bfaa79-0061-439f-b15f-890f354f1940': {'defs': …

In [20]:
plot2 = corr_pipeline.hvplot(   x = 'grenze_starkregen', 
                                y = 'max_Starkregen',
                                by = 'country', 
                                line_width = 2, 
                                title = "Einfluss Grenze(z) auf maximalen Starkregenfall",
                                ylabel= 'Maximaler Starkregen',
                                xlabel= 'Grenze Starkregen(z)')
plot2

BokehModel(combine_events=True, render_bundle={'docs_json': {'c0048f01-3c2d-4816-8e71-280e9a367895': {'defs': …

In [21]:
plot3 = corr_pipeline.hvplot(   x = 'grenze_starkregen', 
                                y = 'min_Starkregen',
                                by = 'country', 
                                line_width = 2, 
                                title = "Einfluss Grenze(z) auf minimalen Starkregenfall",
                                ylabel= 'Einheit [mm/day]',
                                xlabel= 'Grenze Starkregen [z]')
plot3

BokehModel(combine_events=True, render_bundle={'docs_json': {'ae0abb5b-69b2-4b16-b4fa-713052ba5eb8': {'defs': …

In [22]:
plot4 = corr_pipeline.hvplot(   x = 'grenze_starkregen', 
                                y = '%-Strakregen',
                                by = 'country', 
                                line_width = 2, 
                                title = "Einfluss Grenze(z) auf %-Anteil Starkregenfälle")
plot4

BokehModel(combine_events=True, render_bundle={'docs_json': {'4f654f56-38da-414d-b64b-32df483b4f65': {'defs': …

In [23]:
plot5 = corr_pipeline.hvplot(   x = 'grenze_starkregen', 
                                y = 'Starkregen Anzahl',
                                by = 'country', 
                                line_width = 2, 
                                title = "Einfluss Grenze(z) auf Anzahl Starkregenfälle")
plot5

BokehModel(combine_events=True, render_bundle={'docs_json': {'a142ead0-3eb4-469d-bb7f-6bd56fa894d7': {'defs': …

In [24]:
plot6 = corr_pipeline.hvplot(   x = 'grenze_starkregen', 
                                y = 'r-value',
                                by = 'country', 
                                line_width = 2, 
                                title = "Einfluss Grenze(z) auf Korrelation Starkregenfälle zu Agrikultur (r-value)",
                                ylabel= 'R-Wert',
                                xlabel= 'Grenze Starkregen(z)')
plot6

BokehModel(combine_events=True, render_bundle={'docs_json': {'7f7388ad-fe12-4f85-80ea-dbf126f8defd': {'defs': …

In [25]:
plot7 = vals2_pipeline.hvplot(  x = 'year', 
                                y = 'production [1000t]',
                                by = 'country', 
                                line_width = 2, 
                                title = "Produktion über Jahre mit Trend (lineare Regression)",
                                ylabel= 'Produktion[1000t]',
                                xlabel= '')
plot7

BokehModel(combine_events=True, render_bundle={'docs_json': {'8bb1cf5b-7936-436f-a4df-fd4585a042ab': {'defs': …

In [26]:
plot8 = vals2_pipeline.hvplot(  x = 'year', 
                                y = 'fit',
                                by = 'country', 
                                line_width = 2, 
                                title = "Produktion über Jahre mit Trend (lineare Regression)",
                                ylabel= 'Trend der Produktion',
                                xlabel= '')
plot8

BokehModel(combine_events=True, render_bundle={'docs_json': {'d6411ada-6faf-43f7-8443-b7b65ff2f081': {'defs': …

In [27]:
plot9 = vals2_pipeline.hvplot(  x = 'year', 
                                y = 'zdetrend',
                                by = 'country', 
                                line_width = 2, 
                                title = "Produktion über Jahre z-standardiesiert, detrendet",
                                ylabel= 'Z-Detrend',
                                xlabel= '')
plot9

BokehModel(combine_events=True, render_bundle={'docs_json': {'62f9a9e1-6865-482b-a32f-2732212e9aaf': {'defs': …

In [28]:
plot10 = vals2_pipeline.hvplot(  x = 'year', 
                                y = 'Produktion z(detrend)',
                                by = 'country', 
                                line_width = 2, 
                                title = "Produktion über Jahre z-standardiesiert, detrendet",
                                ylabel= 'Produktion detrendet',
                                xlabel= '')
plot10

BokehModel(combine_events=True, render_bundle={'docs_json': {'08a3e095-7e71-4eb0-affa-55daec094530': {'defs': …

In [29]:
plot11 = vals_pipeline.hvplot( x = 'country', 
                                y = radio_group,
                                line_width = 2,
                                kind = 'bar', 
                                title = "Plot für Karte") 
plot11

BokehModel(combine_events=True, render_bundle={'docs_json': {'4e00919e-4620-4b4d-bd76-8c12d075ee5c': {'defs': …

In [30]:
plot12 = vals2_pipeline.hvplot( x = 'year', 
                                y = 'yearly_sum',
                                by = 'country',
                                ylabel= 'Magnitude',
                                line_width = 2,
                                title = "Starkregen Magnitude pro Jahr") 
plot12

BokehModel(combine_events=True, render_bundle={'docs_json': {'ee36d9e2-a391-4f28-a8cd-2a6903c40063': {'defs': …

In [31]:
agriculture_plot = agriculture_pipeline.hvplot(  x = 'year', 
                                        y='production [1000t]',
                                        by='country', 
                                        line_width=2, 
                                        title="Landwirtschaftsertrag pro Jahr",
                                        ylabel= 'Produktion [1000t]',
                                        xlabel= '').opts(xrotation = 35)
agriculture_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'cd214acf-97fa-4b5a-baa5-7c2a26e959cd': {'defs': …

In [32]:
rain_plot = rain_pipeline_yearly.hvplot(  x = 'year', 
                                        y='measurement',
                                        by='country', 
                                        line_width=2, 
                                        title="Regenrate pro Jahr",
                                        ylabel= 'Einheit [kg2/m2/s]',
                                        xlabel= '')
rain_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'e5fd1d41-81a4-47f4-b1da-223b87c7b13c': {'defs': …

# Datastory

In [33]:
width = 900

def textblock(text, title = "", subtitle = ""):
    return (
        f"""
        \n### {title}
        \n#### {subtitle}
        \n {text}
        """
    )

#### Folgen Ackerbau

In [34]:
# Folgen von Starkregen auf den Ackerbau
folgen_ackerbau_0 = textblock(
    title = "",
    subtitle = "",
    text = """Starkregen kann verehrende folgen auf die Landwirtschaft haben. In den folgenden Kapiteln sind ein paar Beispiele beschrieben. Um diesen Einfluss zu untersuchen haben wir den Einfluss von Starkregen auf die Landwirtschaftsproduktion untersucht.
""")
folgen_ackerbau_1 = textblock(
    title = "Reifes Getreide wird zerstört",
    subtitle = "",
    text = """Enormer Regen kann für reife Getreidefelder sehr ungünstig werden. Durch grosse Niederschlagsmengen knickt der Grossteil des Getreides ab, da es auf den Boden gedrückt wird. Durch dieses Geschehen wird der weitere Reifungsprozess stark verlangsamt. Nebenbei wir die Ernte mit dem Mähdrescher erschwert und der Pilzbefall am Korn gefördert. Folgen davon sind klar, Qualitäts- und Ertragsverluste.
""")
folgen_ackerbau_2 = textblock(
    title = "Pfützen und Pflanzen",
    subtitle = "",
    text = """Die folgen von Starkregen sind pro Jahreszeit unterschiedlich. Beziehen wir uns auf das Frühjahr, wo die Pflanzen noch gering entwickelt sind. Starkregen kann hierbei die Bodenstruktur beschädigen. Vor allem betroffen sind Ton und Lehmböden. Die übermässig aufprallenden Tropfen sorgen dafür, dass der Boden verschlämmt. Sobald der Boden verschlammt ist, kann er nur noch eine geringe Menge an Wasser aufnehmen. Hört der Regen hier nicht auf, werden Pfützen gebildet. Anhand dieser Pfützen erhalten die Pflanzenwurzeln nicht mehr genügen Luft. Dies führt zum Absterben dieser. Nebenbei fangen durch Pfützen Pflanzenteile an zu faulen. Konsequenzen davon sind massive Ernteausfälle.
""")
folgen_ackerbau_3 = textblock(
    title = "Schutz vom Boden",
    subtitle = "",
    text = """Viele Schutzmöglichkeiten für die Landwirte gibt es leider nicht. Eine Option wäre es, sich gegen Ernteausfälle zu Versichern. Weiter könnte man den Acker schützen indem man das Feld mit organischem Material. Damit ist in etwa Stallmist oder Kompost gemeint. Dadurch kann der Boden die Wasserspeicherfähigkeit erhöhen.
""")


card1 = pn.Card(    folgen_ackerbau_0,
                    folgen_ackerbau_1,
                    folgen_ackerbau_2,
                    folgen_ackerbau_3,

                    title='Folgen von Starkregen auf den Ackerbau', 
                    background='WhiteSmoke', 
                    sizing_mode='stretch_both', 
                    min_width = 900, margin=5)
card1

BokehModel(combine_events=True, render_bundle={'docs_json': {'02ee3f00-0f04-4dcb-b0df-9a8e41c76ef3': {'defs': …

Card(background='WhiteSmoke', min_width=900, sizing_mode='stretch_both', title='Folgen von Starkregen a...)
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Markdown(str)
    [3] Markdown(str)

#### Landwirtschaft

In [35]:
# Landwirtschaft
landwirtschaft_1 = textblock(
    title = "",
    subtitle = "",
    text = """Bei unseren Landwirtschaftsdaten haben wir bemerkt, dass diese üblicherweise einem Trend folgen (siehe rechts im Bild). Dieser könnte durch eine oder mehrere der folgenden Variablen verursacht werden:
\n•	Politische Entscheide 
\n•	Technischer Fortschritt in der Landwirtschaft
\n•	Verlagerung von Landwirtschaftsproduktion aufgrund von Globalisierung
\nDieser Trend beeinflusst unsere Landwirtschaftsdaten und ist somit eine Störgrösse bei unserer Zusammenhangsanalyse, welche wir später durchführen wollen. Um diesen Trend zu eliminieren haben wir eine Lineare Gerade auf die Landwirtschaftsdaten gefittet. Anschliessend haben wir die Residuen ausgerechnet und diese später für die Zusammenhangsanalyse verwendet. Die Residuen sind die Grösse dafür, wie stark ein Jahr vom Langzeit-Trend abweicht. Also können wir anhand dieser bestimmen ob ein Jahr speziell gut oder schlecht war im Vergleich zum Trend. Schlussendlich haben wir diese Residuen z-standardisiert, so dass wir später bei unserer Korrelationsanalyse nicht nur einzelne Länder auf Korrelation überprüfen können, sondern alle Daten von allen Ländern in einer gemeinsamen Analyse verwenden können (Somit wird die Signifikanz verbessert durch eine grössere Stichprobengrösse). In den folgenden zwei Plots ist dargestellt, wie die Gerade auf die Landwirftschaftsdaten gefittet wird und im zweiten Plot wie die z-standardisierten Residuen dazu aussehen. Wie schon bei der Einleitung erwähnt, hast du links in der Sidebar unter Einstellungen, die Möglichkeit zusätzliche Länder einzublenden.

    
"""
)

card3= pn.Card(     landwirtschaft_1, 
                    (plot8*plot7).panel(width=width), 
                    (plot10*plot9).panel(width=width),
                    title='Aufbereitung Landwirftschaftsdaten', 
                    background='WhiteSmoke', 
                    sizing_mode='stretch_both', 
                    min_width = 900, margin=5)
card3

BokehModel(combine_events=True, render_bundle={'docs_json': {'0ef36558-0614-43e3-a5ef-f49ada8463db': {'defs': …

Card(background='WhiteSmoke', min_width=900, sizing_mode='stretch_both', title='Aufbereitung L...)
    [0] Markdown(str)
    [1] ParamFunction(function, width=900)
    [2] ParamFunction(function, width=900)

#### Definition Starkregen

In [36]:
# Definition Starkregen
definition_starkregen_1 = textblock(
    title = "",
    subtitle = "",
    text = """Wir wollten in dieser Challenge die Auswirkung von Extremereignissen auf die Landwirtschaftliche Produktion untersuchen. Dabei hatten wir zwei Probleme mit der Definition vom Starkregen, welche in den folgenden Kapiteln beschreiben sind:
\n•	Eigentliche Definition eines Extremereignisses
\n•	Auslösung der Daten
""")
definition_starkregen_2 = textblock(
    title = "Problem Definition Extremereignis",
    subtitle = "",
    text = """Wir haben im Internet nach einer für uns geeigneten Definition von Starkregen gesucht. Folgendes war uns dabei wichtig:
\n•	Es muss eine Magnitude berechnet werden, also einen idealerweise metrischen Wert, welcher angibt, wie stark der Starkregen war. Dies brauchen wir, damit wir später den Starkregen auf Korrelation mit den Landwirtschaftsdaten überprüfen können.
\n•	Es muss eine örtliche Abhängigkeit gegeben sein. Dies weil Starkregen nicht universell mit einer festen Grenze definiert werden kann. Z.B: regnet es in England viel häufiger und schwerer als in einem wärmeren Land, wie z.B. Portugal.
\n•	Es muss eine zeitliche Abhängigkeit gegeben sein. Dies weil es in vielen Regionen z.B: im Herbst viel mehr regnet als im Sommer, oder z.B. in Valencia einen Jährlichen Starkregen gibt, welcher immer ca. zur gleichen Zeit eintritt. Das Ereignis ist dann zwar ein Starkregen, für uns jedoch eine Störgrösse, weil das Ereignis Jährlich wiederkehrend ist und somit immer den gleichen Einfluss auf die Agrikultur hat.
\nIm Internet haben wir keine für uns passende Skala gefunden. Jedes Land hat eigene Grenzen festgelegt, ab wann die Regenfälle als Starkregen gelten. Wir mussten also eine eigene Definition finden.
""")
definition_starkregen_3 = textblock(
    title = "Problem Auflösung der Daten",
    subtitle = "",
    text = "Zudem hatten wir das Problem, dass unsere Rohdaten vom Starkregen nicht direkt vergleichbar waren mit den Werten von der Agrikultur. Dies hing davon ab, dass unsere Regendaten viel höherer aufgelöst waren als unsere Agrikulturwerte:"
)
definition_starkregen_4 = textblock(
    title = "",
    subtitle = "",
    text = "Wir mussten also auch beachten, dass bei unserer Definition von Starkregen ein Jahreswert entsteht, welcher in der gleichen Auflösung vorliegt wie die Agrikulturwerte, so dass wir diese vergleichen können."
)
definition_starkregen_5 = textblock(
    title = "Datenreduktion",
    subtitle = "",
    text = """Weil wir enorm viele Daten hatten, haben wir mit unseren Fachexperten abgesprochen, dass wir eine reduzierte Anzahl an Ländern in unserer Analyse beachten. Wir haben also als erstes unsere Daten auf folgende 5 Länder reduziert (Geographische Lage unten im Bild):
\n•	Schweden
\n•	Norwegen
\n•	Italien
\n•	Spanien 
\n•	Portugal
""")
definition_starkregen_7 = textblock(
    title = "Z-Standardisierung",
    subtitle = "",
    text = """Damit wir einen universellen Grenzwert für unsere Regendaten erhalten, welcher örtlich und zeitlich abhängig ist, haben wir unsere Regensummen nach Grid-Unit und Monat gruppiert und innerhalb der Gruppierung eine z-Standardisierung druchgeführt. Mit der z-Standardisierung werden die Niederschlagswerte unter den einzelnen Grid-units und monatsweise vergleichbar untereinander. Somit können wir dann eine Magnitude berechnen (rechts im Bild gezeigt):
\nMagnitude= ∆(z)-Grenze(z)
\nWenn wir nun diese Magnituden pro Grid-Unit Jährlich aufsummieren, so erhalten wir pro Grid-Unit eine Jahresmagnitude (Beispielhaft unten im Bild gezeigt). Anschliessend haben wir den Mittelwert der Jahresmagnitude der Grid-Units welchem einem Land zugewiesen sind, als Jahresmagnitude / Land genommen.
"""
)
definition_starkregen_8 = textblock(
    title = "",
    subtitle = "",
    text = "In dem folgenden Plot sind die Magnituden pro Jahr dargestellt, welche wir erhalten haben:"
)
definition_starkregen_9 = textblock(
    title = "Eigentlicher Niederschlag",
    subtitle = "",
    text = """ Bei unserer Definition und der Berechnung der Magnitude werden nur Z-Werte beachtet und die x% mit dem stärksten Ausschlag. Was dabei nicht beachtet wird, welchen eigentlichen Niederschlagswert vorliegt (in [mm/m3 day]). So kann es bei unserer Definition sein, dass wir einen Starkregen haben von nur z.B. 50 [mm/m3 day]). Durch diese Art von Starkregen wird die Landwirtschaft aber nicht negativ beeinflusst.
Um dies etwas analysieren zu können haben wir drei Plots erstellt. Auf den Plots ist zu sehen, wie sich die definierte Grenze vom Starkregen auf die eigentlichen Niederschlagswerte, der zur Analyse verwendeten Daten auswirkt. Wir haben dabei die folgenden drei Kennwerte überprüft:
\n•	Maximaler Niederschlag
\n•	Minimaler Niederschlag
\n•	Durchschnittlicher Niederschlag
\nWas in dem Plot zu erkennen ist, dass ab einer gewissen Grenze(z) der Maximale Niederschlag in machen Ländern sinkt. Dies schein zu Beginn ein Fehler zu sein, jedoch wird dies verursacht dadurch, dass wir die Niederschlagsmengen pro Monat separat über alle Jahre hinweg z-standardisieren. Somit wenn es in diesen Monaten immer speziell stark regnet, dann wird der z-Wert verhältnismässig nicht so gross sein und bei steigender Grenze(z) fallen diese Werte schneller weg. 
\nAn dieser Stelle möchten wir nochmals darauf hinweisen, dass die Plots interaktiv sind. Wir laden dich dazu ein, dein eigenes Land weiter zu analysieren und es evtl. auch mit anderen Ländern zu vergleichen."""
)



card4 = pn.Card(    definition_starkregen_1,
                    definition_starkregen_2,
                    definition_starkregen_3,
                    pn.pane.PNG('./plots/ds_1.png', sizing_mode='scale_width', max_width = 500, height=300),
                    definition_starkregen_4,
                    definition_starkregen_5,
                    pn.pane.PNG('./plots/ds_2.png', sizing_mode='scale_width', max_width = 400, height=300),
                    definition_starkregen_7,
                    pn.pane.PNG('./plots/ds_4.png', sizing_mode='scale_width', max_width = 200, height=300),
                    definition_starkregen_8,
                    plot12.panel(width=width),
                    definition_starkregen_9,
                    plot3.panel(width=width),
                    plot2.panel(width=width),
                    plot1.panel(width=width), 
                    title='Definition Starkregen', 
                    background='WhiteSmoke', 
                    sizing_mode='stretch_both', 
                    min_width = 900, margin=5)
card4

BokehModel(combine_events=True, render_bundle={'docs_json': {'6c3e7cf1-121b-42b6-b00d-d88860f6df85': {'defs': …

Card(background='WhiteSmoke', min_width=900, sizing_mode='stretch_both', title='Definition Starkregen')
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Markdown(str)
    [3] PNG(str, height=300, max_width=500, sizing_mode='scale_width')
    [4] Markdown(str)
    [5] Markdown(str)
    [6] PNG(str, height=300, max_width=400, sizing_mode='scale_width')
    [7] Markdown(str)
    [8] PNG(str, height=300, max_width=200, sizing_mode='scale_width')
    [9] Markdown(str)
    [10] ParamFunction(function, width=900)
    [11] Markdown(str)
    [12] ParamFunction(function, width=900)
    [13] ParamFunction(function, width=900)
    [14] ParamFunction(function, width=900)

#### Korrelationsanalyse

In [37]:
# Korrelationsanalyse
korrelationsanalyse_1 = textblock(
    text = "Um die Extremereignisse auf Zusammenhang zu überprüfen haben wir die Starkregenmagnitude pro Jahr und Land, auf Korrelation mit der Landwirtschafts-Produktion überprüft."
)
korrelationsanalyse_2 = textblock(
    title = "Z-Standardisierung Landwirtschaft",
    text = "Für die Zusammenhangsanalyse haben wir den Detrend (Residuum Trend/Istwert) der Landwirtschafts-Produktion zusätzlich noch z-standardisiert. Dies aus dem Grund, dass so die einzelnen Länder untereinander vergleichbar werden. So konnten wir nicht nur einzelne Länder auf Korrelation mit Extremregen überprüfen, sondern auch eine Korrelationsanalyse mit allen Ländern durchführen. Dies ist in unserem Fall sehr interessant, weil wir nur eine Zeitspanne von 10 Jahren untersucht haben. Somit können wir anstelle der 10 Datenpunkte für ein Land, alle 50 Datenpunkte für alle Länder auf Korrelation überprüfen."
)
korrelationsanalyse_3 = textblock(
    title = "Hypothesen",
    text = """Wir hatten folgende Hypothesen aufgestellt:
\n•	H1: Es gibt einen irgendwie gearteten Zusammenhang zwischen den Extremereignissen (gem. unserer Definition) und der Landwirtschaftsproduktion (z-standardisierte Residuen)
\n•	H0: Es gibt keinen Zusammenhang zwischen den Extremereignissen (gem. unserer Definition) und der Landwirtschaftsproduktion (z-standardisierte Residuen)
\nUnsere Hypothese haben wir ungerichtet formuliert, weil wir nicht genügend Daten/Vorwissen hatten, um gerichtet zu testen. Als Signifikanz Niveau haben wir 5% gewählt, da dies ein übliches Signifikanz Niveau in der Statistik ist.
"""
)
korrelationsanalyse_4 = textblock(
    title = "Ergebnis",
    text = "Wir haben in einem Bereich von 2 bis 10 (80-Einzelwerte) für die Grenze(z) für Starkregen auf Korrelation mit der Landwirtschaftsproduktion getestet und auf ein Signifikanzniveau von 5% überprüft. Wir konnten keine statistisch Signifikate Korrelation zwischen den beiden Variablen finden."
)
korrelationsanalyse_5 = textblock(
    title = "Einfluss Grenze(z) auf Korrelation",
    text = "Wir haben zusätzlich noch den Einfluss von unserer Grenze(z) für den Starkregen auf unsere Analyse die Korrelation untersucht. Dazu haben wir die unterschiedlichen Plots unten erstellt. Auf den Plots ist jeweils der Einfluss der Grenze(z) auf verschiedene Grenzwerte dargestellt:"
)
korrelationsanalyse_6 = textblock(
    subtitle="Plot 1",
    text = "Einfluss auf die Korrelation zwischen Starkregen und Landwirtschaftsertrag. (Negative Korrelation bedeutet, dass sich Starkregen negativ auf Landwirtschaftserträge auswirkt). Es ist zu erkennen, dass die Korrelation bis ca. einem Grenzwert von 4 stetig negativer wird und anschliessend zu rauschen beginnt. Hierbei ist es wichtig anzumerken, dass keines der Korrelationen statistisch signifikat ist. Dies ist auf unsere geringe Stichprobengrösse von 50 Datenpunkten zurückzuführen."
)
korrelationsanalyse_8 = textblock(
    subtitle="Plot 2",
    text = "Nochmals der Plot zum Einfluss der Grenze(z) auf den durchschnittlichen Regenfall. Interessant dabei ist, dass wenn unter den Einstellungen links in der Sidebar, alle Länder eingeblendet werden sieht man, dass Aufgrund der Monatlichen z-Standardisierung der Durchschnittliche Starkregenfall ab einem Grenzwert von ca. 5 bei manchen Ländern abnimmt. Und ab einem Grenzwert von ca. 7.5 nimmt dieser stark ab. Die Korrelation scheint im ähnlichem Ausmass zu schwingen."
)
korrelationsanalyse_9 = textblock(
    subtitle="Plot 3 & 4",
    text = "In diesen Plots ist dargestellt, wie sich die Grenze(z) auf den % Anteil der Regenfälle und der eigentlichen Anzahl der Regenfälle auswirkt. Man kann erkennen, dass der % Anteil bei allen Ländern ca. gleich verläuft, dies ist darauf zurückzuführen, dass wir die Regenfälle pro Land einzeln z-Standardisiert haben. Die eigentlichen Anzahlen sind pro Land jedoch stark unterschiedlich. Die Stetige Abnahme der Anzahl Datenpunkte, welche zur Analyse verwendet werden, tragen direkt zum Rauschen bei der Korrelation bei. Dies ist am Beispiel Portugal klar ersichtlich, da Portugal bei der Korrelation am meisten Rauschen hat und auch am wenigsten eigentliche Regenfälle."
)

card5 = pn.Card(    korrelationsanalyse_1,
                    korrelationsanalyse_2,
                    korrelationsanalyse_3,
                    korrelationsanalyse_4,
                    korrelationsanalyse_5,
                    korrelationsanalyse_6,
                    plot6.panel(width=width),
                    korrelationsanalyse_8,
                    plot1.panel(width=width),
                    korrelationsanalyse_9,
                    plot4.panel(width=width),
                    plot5.panel(width=width),
                    title='Korrelationsanalyse', 
                    background='WhiteSmoke', 
                    sizing_mode='stretch_both', 
                    min_width = 900, 
                    margin=5)
card5

BokehModel(combine_events=True, render_bundle={'docs_json': {'225f41fd-5265-4c8f-824f-d9fb06ee1056': {'defs': …

Card(background='WhiteSmoke', min_width=900, sizing_mode='stretch_both', title='Korrelationsanalyse')
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Markdown(str)
    [3] Markdown(str)
    [4] Markdown(str)
    [5] Markdown(str)
    [6] ParamFunction(function, width=900)
    [7] Markdown(str)
    [8] ParamFunction(function, width=900)
    [9] Markdown(str)
    [10] ParamFunction(function, width=900)
    [11] ParamFunction(function, width=900)

#### Fazit

In [38]:
# Folgen von Starkregen auf den Ackerbau
fazit_0 = textblock(
    title = "",
    subtitle = "",
    text = """Schlussendlich konnten wir keine statistisch relevante Korrelation finden. Jedoch könnten bei einer weiteren Analyse mehr Datenpunkte verwendet werden, somit wäre die Stichprobengrösse grösser und es würden vermutlich statistisch signifikante Resultate entstehen. Generell haben wir aber in unserer Analyse auch festellen müssen, dass eine Analyse auf unsere Art und weise vermutlich nicht der Ideale Weg ist, um einen Zusammenhang zwischen Starkregen und Landwirftschaftserträgen aufzuzeigen. Es wäre vermutlich besser über Europa hinweg verteilt spezifische Bauern genauer zu betrachten. So wären bessere Daten vorhanden und der Zusammenhang könnte genauer untersucht werden.
""")


card6 = pn.Card(    fazit_0,
                    title='Fazit', 
                    background='WhiteSmoke', 
                    sizing_mode='stretch_both', 
                    min_width = 900, margin=5)
card6

BokehModel(combine_events=True, render_bundle={'docs_json': {'d8a0d462-5c3a-439e-9b58-2c7251a4577a': {'defs': …

Card(background='WhiteSmoke', min_width=900, sizing_mode='stretch_both', title='Fazit')
    [0] Markdown(str)

# Dashboard erstellen

In [39]:
#Layout using Template
width = 900
small_width = 650
margin = 0
template = pn.template.FastListTemplate(
    title='Klimadaten Dashboard: Starkregen mit den Auswirkungen auf den Landwirtschaftsertrag',
    sidebar_width = 360,
    sidebar=[   pn.pane.Markdown("## Einstellungen"),
                pn.pane.Markdown("Die Datastory ist mit Plots angereichert, auf welchen wir den Leser einladen die einzelnen interaktiven Plots zu explorieren. Mit dem Widget unten können Sie die Länder auswählen, welche dargestellt werden sollen. Es ist möglich mehrere Länder zeitgleich anzuzeigen."),
                multi_country_selector,
                pn.pane.Markdown("\n\n## Referenzdaten"),
                pn.pane.Markdown("In der folgenden Landkarte sind die wichtigsten Referenzwerte zur Datastory dargestellt: 'Berechnete Magnitude' und 'z-stardardisierteer detrend'. Der Plot ist interaktiv und kann mit den folgenden Widgets gefiltert werden"),       
                pn.Column(grenze_slider),  
                pn.Column(radio_group),
                pn.pane.PNG('./plots/europe_png.png', sizing_mode='scale_width', max_width = 325, height=250)],
    main=[  pn.Row(pn.Column(   "# Datastory",
                                "In folgenden Abschnitten werden wir euch versuchen unsere Challengen näherzubringen. Wir werden disskutieren, wie wir den Einfluss der Extremniederschläge auf die Landwirftschaftsdaten untersucht haben. Die Datastory ist mit Plots angereichert, auf welchen wir den Leser einladen die einzelnen interaktiven Plots zu explorieren. Links in der Sidebar unter Einstellungen können Sie die Länder auswählen, welche dargestellt werden sollen. Es ist möglich mehrere Länder zeitgleich anzuzeigen.",
                                card1,
                                card4,
                                card3,
                                card5,
                                card6)),
            pn.Row( pn.Column(  "# Dashboard",
                                pn.Card(
                                "### Einstellungen Dashboard:",
                                multi_country_selector2,
                                start_year_slider,
                                end_year_slider,    
                                rain_plot.panel(sizing_mode = 'stretch_width', min_width = 900),    
                                agriculture_plot.panel(sizing_mode = 'stretch_width', min_width = 900),    
                                title = "Dashboard",
                                background='WhiteSmoke', 
                                sizing_mode='stretch_both', 
                                min_width = 900, 
                                margin=5)))])

# template.show()
template.show();

Launching server at http://localhost:54522
